## Exercise 1: Creating class, attributes, methods with Online Shopping Cart

In [38]:
class Shopping:
    def __init__(self, name, cart, price_list,balance = 0):
        self.name = name
        self.cart = cart
        self.balance = balance
        self.price_list = price_list
        
    def check_balance(self):
        print(f"Your current balance is {self.balance} Eur")
    
    def remove_item(item = None):
        try:
            self.cart.remove(item)
            
        except:
            print(f'There is no {item} in selected goods')

In [39]:
rita = Shopping('Rita', ['banana', 'apple'], [10, 5])

In [40]:
rita.price_list

[10, 5]

In [41]:
rita.check_balance()

Your current balance is 0 Eur


In [42]:
rita.remove_item('banana')

TypeError: remove_item() takes from 0 to 1 positional arguments but 2 were given

## Exercise 2: Multiple Inheritance and mro with Mendel's Dominant and Recessive Traits Theory

In [45]:
class Dominant:
    def __init__(self, seed_shape = 'round', pod_color = 'green', 
                 flower_color = 'white'):
        self.seed_shape = seed_shape
        self.pod_color = pod_color
        self.flower_color = flower_color

In [49]:
class Recessive:
    def __init__(self, seed_shape = 'wrinkled', pod_color = 'yellow', 
                 flower_color = 'white'):
        self.seed_shape = seed_shape
        self.pod_color = pod_color
        self.flower_color = flower_color

In [82]:
class Pea( Dominant, Recessive):
    def __init__(self):
        super().__init__()
    


In [83]:
pea = Pea()

In [84]:
pea.pod_color

'green'

In [85]:
pea.seed_shape

'round'

In [86]:
pea.flower_color

'white'

In [88]:
print(Pea.__mro__) == print(Pea.mro())

(<class '__main__.Pea'>, <class '__main__.Dominant'>, <class '__main__.Recessive'>, <class 'object'>)
[<class '__main__.Pea'>, <class '__main__.Dominant'>, <class '__main__.Recessive'>, <class 'object'>]


True

### Exercise 4: OOP with application to scVI

In [99]:
class Trainer:
    r"""The abstract Trainer class for training a PyTorch model and monitoring its statistics. It should be
    inherited at least with a .loss() function to be optimized in the training loop.
    Args:
    :model: A model instance from class ``VAE``, ``VAEC``, ``SCANVI``
    :gene_dataset: A gene_dataset instance like ``CortexDataset()``
    :use_cuda: Default: ``True``.
    :metrics_to_monitor: A list of the metrics to monitor. If not specified, will use the
    ``default_metrics_to_monitor`` as specified in each . Default: ``None``.
    :benchmark: if True, prevents statistics computation in the training. Default: ``False``.
    :verbose: If statistics should be displayed along training. Default: ``None``.
    :frequency: The frequency at which to keep track of statistics. Default: ``None``.
    :early_stopping_metric: The statistics on which to perform early stopping. Default: ``None``.
    :save_best_state_metric: The statistics on which we keep the network weights achieving the best store, and
    restore them at the end of training. Default: ``None``.
    :on: The data_loader name reference for the ``early_stopping_metric`` and ``save_best_state_metric``, that
    should be specified if any of them is. Default: ``None``.
    """
    default_metrics_to_monitor = []
    def __init__(self, model, gene_dataset, use_cuda=True, metrics_to_monitor=None, benchmark=False,
                 verbose=False, frequency=None, weight_decay=1e-6, early_stopping_kwargs=dict(),
                 data_loader_kwargs=dict()):
        self.model = model
        self.gene_dataset = gene_dataset
        self.data_loader_kwargs = {
        "batch_size": 128,
        "pin_memory": use_cuda
        }
        self.data_loader_kwargs.update(data_loader_kwargs)
        self.weight_decay = weight_decay
        self.benchmark = benchmark
        self.epoch = -1 # epoch = self.epoch + 1 in compute metrics
        self.training_time = 0
        self.early_stopping = EarlyStopping(**early_stopping_kwargs)
        
        def __getattr__(self, name):
            if '_posteriors' in self.__dict__:
                _posteriors = self.__dict__['_posteriors']
                if name.strip('_') in _posteriors:
                    return _posteriors[name.strip('_')]
            return object.__getattribute__(self, name)
        def __delattr__(self, name):
            if name.strip('_') in self._posteriors:
                del self._posteriors[name.strip('_')]
            else:
                object.__delattr__(self, name)
                
        def register_posterior(self, name, value):
            name = name.strip('_')
            self._posteriors[name] = value
            
        def corrupt_posteriors(self, rate=0.1, corruption="uniform", update_corruption=True):
            if not hasattr(self.gene_dataset, 'corrupted') and update_corruption:
                self.gene_dataset.corrupt(rate=rate, corruption=corruption)
            for name, posterior in self._posteriors.items():
                self.register_posterior(name, posterior.corrupted())

1. What are the arguments with default values stated in the Trainer class i.e. name the arguments which are not necessary to include
while instantiating the Trainer class?

2. Can you name at least 2 model names the user can use as arguments while instantiating the class Trainer? e.g:
train_object=Trainer(model=?,gene_dataset,...)

train1 = Trainer('VAE', ...) ;
train2 = Trainer('SCANVI', ...

3. Can you name two arguments where you can use variable length of the arguments while instantiating the Trainer class?

early_stopping_kwargs=dict(),
data_loader_kwargs=dict())

4. List all the methods that you can see under Trainer class. Which of these are Dunder methods?

__getattr__, __delattr__

## Example 4.2

In [107]:
class UnsupervisedTrainer(Trainer):
    r"""The VariationalInference class for the unsupervised training of an autoencoder.
    Args:
    :model: A model instance from class ``VAE``, ``VAEC``, ``SCANVI``
    :gene_dataset: A gene_dataset instance like ``CortexDataset()``
    :train_size: The train size, either a float between 0 and 1 or and integer for the number of training samplesto use Default: ``0.8``.
    :\*\*kwargs: Other keywords arguments from the general Trainer class.
    Examples:
    >>> gene_dataset = CortexDataset()
    >>> vae = VAE(gene_dataset.nb_genes, n_batch=gene_dataset.n_batches * False,
    ... n_labels=gene_dataset.n_labels)
    >>> infer = VariationalInference(gene_dataset, vae, train_size=0.5)
    >>> infer.train(n_epochs=20, lr=1e-3)
    """
    default_metrics_to_monitor = ['ll']
    
    def __init__(self, model, gene_dataset, train_size=0.8, test_size=None, kl=None, **kwargs):
        super().__init__(model, gene_dataset, **kwargs)
        self.kl = kl
        if type(self) is UnsupervisedTrainer:
            self.train_set, self.test_set = self.train_test(model, gene_dataset, train_size, test_size)
            self.train_set.to_monitor = ['ll']
            self.test_set.to_monitor = ['ll']
    @property
    def posteriors_loop(self):
        return ['train_set']
    
class AdapterTrainer(UnsupervisedTrainer):
    def __init__(self, model, gene_dataset, posterior_test, frequency=5):
        super().__init__(model, gene_dataset, frequency=frequency)
        self.test_set = posterior_test
        self.test_set.to_monitor = ['ll']
        self.params = list(self.model.z_encoder.parameters()) + list(self.model.l_encoder.parameters())
    
    @property
    def posteriors_loop(self):
        return ['test_set']
    
    def train(self, n_path=10, n_epochs=50, **kwargs):
        for i in range(n_path):
            # Re-initialize to create new path
            self.model.z_encoder.load_state_dict(self.z_encoder_state)
            self.model.l_encoder.load_state_dict(self.l_encoder_state)
            super().train(n_epochs, params=self.params, **kwargs)
        return min(self.history["ll_test_set"])

Answer the following questions:
1. Which class does UnsupervisedTrainer inherit from?
2. If we print the AdapterTrainer.__mro__, what order of inheritance would you see?
3. Look at the posteriors_loop method for the class AdapterTrainer. Modify the code such that now we donot want this to be the
attribute of the class, but rather as a method which takes in argument set and returns this value and default it to ['test_set']